In [1]:
#import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

from numpy import array , hstack
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Activation
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError as mse
from keras.metrics import RootMeanSquaredError,accuracy
from keras.optimizers import Adam

In [2]:
df = pd.read_csv(r"C:\Users\Acer\Desktop\Model_creation\dataset\City_day01.csv")
df = df[df['City']=='Delhi']
df.index= pd.to_datetime(df['Date'])
df = df.drop(['City','Date','NO','NOx','Benzene','Toluene','Xylene'],axis = 1)

In [3]:
from sklearn import preprocessing
Scaler = preprocessing.MinMaxScaler()
for i in df.columns:
    df[[i]] = Scaler.fit_transform(df[[i]])


In [4]:
def split_sequences(sequences, n_steps_in, n_steps_out):
        X = []
        y = []
        sequences = sequences.to_numpy()
        for i in range(len(sequences)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out-1
            # check if we are beyond the dataset
            if out_end_ix > len(sequences):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

In [5]:
n_steps_in, n_steps_out = 60 , 30 # choose a number of time steps #change this accordingly
X, y = split_sequences(df, n_steps_in, n_steps_out) # covert into input/output
print ("X.shape" , X.shape) 
print ("y.shape" , y.shape)     

X.shape (1921, 60, 7)
y.shape (1921, 30)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

print('X_train',X_train.shape), print('y_train',y_train.shape)
print('X_test',X_test.shape), print('y_test',y_test.shape)
n_features = X_train.shape[2]
print('n_features=',n_features)

X_train (1536, 60, 7)
y_train (1536, 30)
X_test (385, 60, 7)
y_test (385, 30)
n_features= 7


In [7]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            11600     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 30)                1530      
                                                                 
 activation (Activation)     (None, 30)                0         
                                                                 
Total params: 33,330
Trainable params: 33,330
Non-trainable params: 0
_________________________________________________________________


In [8]:
cp  = ModelCheckpoint('model/',save_best_only=True)
# cp = [
#     keras.callbacks.EarlyStopping(monitor='val_loss',
#                                   mode='min',
#                                   patience=1,
#                                   verbose=1,
#                                   restore_best_weights=True)]
model.compile(loss=mse(), optimizer=Adam(learning_rate=0.0001), metrics=['RootMeanSquaredError'])

In [9]:
history  = model.fit(X_train,y_train, validation_data=(X_test, y_test),batch_size=25,shuffle=False,epochs=55, callbacks=[cp])

Epoch 1/55
62/62 [==============================] - 31s 421ms/step - loss: 0.1791 - root_mean_squared_error: 0.4232 - val_loss: 0.1540 - val_root_mean_squared_error: 0.3924
Epoch 2/55
62/62 [==============================] - 25s 403ms/step - loss: 0.1162 - root_mean_squared_error: 0.3409 - val_loss: 0.0856 - val_root_mean_squared_error: 0.2926
Epoch 3/55
62/62 [==============================] - 25s 401ms/step - loss: 0.0640 - root_mean_squared_error: 0.2530 - val_loss: 0.0509 - val_root_mean_squared_error: 0.2257
Epoch 4/55
62/62 [==============================] - 26s 416ms/step - loss: 0.0409 - root_mean_squared_error: 0.2021 - val_loss: 0.0373 - val_root_mean_squared_error: 0.1931
Epoch 5/55
62/62 [==============================] - 25s 398ms/step - loss: 0.0319 - root_mean_squared_error: 0.1785 - val_loss: 0.0320 - val_root_mean_squared_error: 0.1788
Epoch 6/55
62/62 [==============================] - 28s 453ms/step - loss: 0.0286 - root_mean_squared_error: 0.1690 - val_loss: 0.0299 

Epoch 27/55
62/62 [==============================] - 28s 448ms/step - loss: 0.0191 - root_mean_squared_error: 0.1383 - val_loss: 0.0193 - val_root_mean_squared_error: 0.1388
Epoch 28/55
62/62 [==============================] - 27s 443ms/step - loss: 0.0189 - root_mean_squared_error: 0.1375 - val_loss: 0.0190 - val_root_mean_squared_error: 0.1378
Epoch 29/55
62/62 [==============================] - 27s 444ms/step - loss: 0.0187 - root_mean_squared_error: 0.1368 - val_loss: 0.0187 - val_root_mean_squared_error: 0.1366
Epoch 30/55
62/62 [==============================] - 28s 455ms/step - loss: 0.0185 - root_mean_squared_error: 0.1359 - val_loss: 0.0183 - val_root_mean_squared_error: 0.1354
Epoch 31/55
62/62 [==============================] - 26s 423ms/step - loss: 0.0182 - root_mean_squared_error: 0.1349 - val_loss: 0.0180 - val_root_mean_squared_error: 0.1341
Epoch 32/55
62/62 [==============================] - 27s 437ms/step - loss: 0.0179 - root_mean_squared_error: 0.1339 - val_loss: 0

Epoch 53/55
62/62 [==============================] - 27s 438ms/step - loss: 0.0146 - root_mean_squared_error: 0.1209 - val_loss: 0.0146 - val_root_mean_squared_error: 0.1208
Epoch 54/55
62/62 [==============================] - 29s 465ms/step - loss: 0.0145 - root_mean_squared_error: 0.1205 - val_loss: 0.0144 - val_root_mean_squared_error: 0.1200
Epoch 55/55
62/62 [==============================] - 29s 469ms/step - loss: 0.0144 - root_mean_squared_error: 0.1200 - val_loss: 0.0143 - val_root_mean_squared_error: 0.1194


In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots()
# Add traces
fig.add_trace(
    go.Scatter( y=history.history['val_loss'], name="val_loss"),
    secondary_y=False,)
fig.add_trace(
    go.Scatter( y=history.history['loss'], name="loss"),
    secondary_y=False,)
# Set x-axis title
fig.update_xaxes(title_text="Epoch")
# Set y-axis title
fig.update_yaxes(title_text="Loss")

NameError: name 'history' is not defined

In [12]:
from tensorflow.keras.models import load_model
model = load_model('model/')

In [13]:
# #save model inferences
model.save('Delhi.h5')

In [24]:
data = pd.read_csv(r"C:\Users\Acer\Desktop\Model_creation\dataset\test.csv")
data.index= pd.to_datetime(data['Date'])
data = data.drop(['Date'],axis = 1)


In [25]:
from sklearn import preprocessing
Scaler = preprocessing.MinMaxScaler()
for i in data.columns:
    data[[i]] = Scaler.fit_transform(data[[i]])


In [26]:
test = data.drop(['AQI'],axis = 1)
test = test.to_numpy()
test = test.reshape(1,test.shape[0],test.shape[1])
test.shape

(1, 60, 7)

In [27]:
y_pred = model.predict(test)
y_pred_inv = Scaler.inverse_transform(y_pred)
y_pred_inv = y_pred_inv.reshape(-1,1)

1/1 [==============================] - 0s 88ms/step


In [28]:
import plotly.express as px
fig = px.line(y_pred_inv,  title='30 day forecaste')
fig.show()

In [29]:
from tensorflow.keras.models import load_model
model1 = load_model('Delhi.h5')

In [30]:
data = pd.read_csv(r"C:\Users\Acer\Desktop\Model_creation\dataset\test.csv")
data.index= pd.to_datetime(data['Date'])
data = data.drop(['Date'],axis = 1)
from sklearn import preprocessing
Scaler = preprocessing.MinMaxScaler()
for i in data.columns:
    data[[i]] = Scaler.fit_transform(data[[i]])
test = data.drop(['AQI'],axis = 1)
test = test.to_numpy()
test = test.reshape(1,test.shape[0],test.shape[1])
test.shape

(1, 60, 7)

In [31]:
y_pred = model1.predict(test)
y_pred_inv = Scaler.inverse_transform(y_pred)
y_pred_inv = y_pred_inv.reshape(-1,1)

1/1 [==============================] - 0s 455ms/step


In [32]:
import plotly.express as px
fig = px.line(y_pred_inv,  title='30 day forecaste')
fig.show()